In [1]:
import pandas as pd
import numpy as np 

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from datetime import datetime
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(1)
tf.set_seed = 42


# train_feature = np.load('Data/defog/train_feature.npy')
# valid_feature = np.load('Data/defog/valid_feature.npy')
# test_feature = np.load('Data/defog/test_feature.npy')
# train_label = np.load('Data/defog/train_label.npy')
# valid_label = np.load('Data/defog/valid_label.npy')
# test_label = np.load('Data/defog/test_label.npy')


2023-05-08 16:26:24.412813: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# train_feature.shape, valid_feature.shape, train_label.shape, valid_label.shape, test_feature.shape, test_label.shape

In [3]:
# # Building the model and pipeline for Tensorflow 
# # Preprocess the data
# # Turn our data into TensorFlow Datasets
# train_dataset = tf.data.Dataset.from_tensor_slices((train_feature, train_label))
# train_dataset =  train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

# valid_dataset = tf.data.Dataset.from_tensor_slices((valid_feature,valid_label))
# valid_dataset = valid_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

# test_dataset = tf.data.Dataset.from_tensor_slices((test_feature,test_label))
# test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

In [4]:
# path = 'Data/defog/'
# train_dataset.save(path+'train_dataset')
# valid_dataset.save(path+'valid_dataset')
# test_dataset.save(path+'test_dataset')

In [5]:
path = 'Data/defog/'
train_dataset = tf.data.Dataset.load(path+'train_dataset')
valid_dataset = tf.data.Dataset.load(path+'valid_dataset')
test_dataset =  tf.data.Dataset.load(path+'test_dataset')
test_feature = np.load('Data/defog/test_feature.npy')
test_label = np.load('Data/defog/test_label.npy')

In [6]:
# Check the data shape
# print(
#     f"Train_feature : {train_feature.shape} {train_feature.dtype} \n" 
#     f"Train_labels : {train_label.shape} {train_label.dtype} \n" 
#     f"Valid_feature : {valid_feature.shape} {valid_feature.dtype} \n" 
#     f"Valid_label : {valid_label.shape} {valid_label.dtype} "
#     ) 

print(f"Train : {train_dataset} \n"
      f"Valid : {valid_dataset} \n" 
      f"Test : {test_dataset}")

Train : <_LoadDataset element_spec=(TensorSpec(shape=(None, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 4), dtype=tf.int64, name=None))> 
Valid : <_LoadDataset element_spec=(TensorSpec(shape=(None, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 4), dtype=tf.int64, name=None))> 
Test : <_LoadDataset element_spec=(TensorSpec(shape=(None, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 4), dtype=tf.int64, name=None))>


In [7]:
# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=5) # if val loss decreases for 3 epochs in a row, stop training

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",  
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=3,
                                                 verbose=1, # print out when learning rate goes down 
                                                 min_lr=1e-7)

In [8]:
# Build the model 
epoch = 25
input_shape = 3

inputs = layers.Input(shape=input_shape, name="input_layer")
x = layers.Dense(16, activation ='elu')(inputs)
x = layers.Dense(32, activation = 'elu')(x)
# x = layers.BatchNormalization()(x)
x = layers.Dropout(0.1)(x)
# x = layers.Dense(64, activation ='elu')(x)
# x = layers.Dense(128, activation = 'elu')(x)
# # x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.25)(x)
# x = layers.Dense(128, activation ='elu')(x)
# x = layers.Dense(64, activation = 'elu')(x)
# # x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Flatten()(x)
x = layers.Dense(32, activation="elu")(x)
x = layers.Dense(16, activation="elu")(x)
outputs = layers.Dense(4, activation="softmax",name="output_layer")(x)      
model = tf.keras.Model(inputs, outputs) 

model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

model.summary()



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 3)]               0         
                                                                 
 dense (Dense)               (None, 16)                64        
                                                                 
 dense_1 (Dense)             (None, 32)                544       
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 64)                2112      
                                                                 
 dense_3 (Dense)             (None, 128)               8320      
                                                                 
 dropout_1 (Dropout)         (None, 128)               0     

In [9]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import classification_report

In [10]:
import numpy as np
np.random.seed(40)
from mlflow import MlflowClient
from mlflow.entities import ViewType
from mlflow.models.signature import infer_signature

# !mlflow server --backend-store-uri sqlite:///backend.db

In [11]:
def eval_metrics(classifier, test_features, test_labels):
    
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support
    
    # make prediction
    model_preds_probs   = classifier.predict(test_features)
    model_preds         = tf.argmax(model_preds_probs, axis=1)
    predictions         = tf.argmax(test_labels,axis=1)

    
    # Calculate model accuracy
    model_accuracy = accuracy_score(test_labels, predictions) * 100
    # Calculate model precision, recall and f1 score using "weighted average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(test_labels, 
                                                                                 predictions, 
                                                                                 average="weighted", 
                                                                                 zero_division= 1)
    
    av_precision    = average_precision_score(test_labels, predictions, zero_division= 1)
    
    model_results = {
        "accuracy": model_accuracy,
        "precision": model_precision,
        "recall": model_recall,
        "f1": model_f1,
        "av precisoin" : av_precision}
    
    
    
    
    
    target_names = ['StartHesitation','Turn','Walking', 'All_zero']
    print("Classification report")
    print("---------------------","\n")
    print(classification_report(test_labels, predictions,target_names=target_names),"\n")
    # print("Confusion Matrix")
    # print("---------------------","\n")
    # print(f"{Matrix} \n")

    print("Accuracy Measures")
    print("---------------------","\n")
    print("Accuracy: ", model_accuracy)
    print("Avarge Precision: ", av_precision)
    print("precision: ", model_precision)
    print("recall : ", model_recall)
    print("f1 : ", model_f1)
    
    return model_results

In [12]:
import mlflow
import mlflow.tensorflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")


tracking URI: 'http://127.0.0.1:5000'


In [14]:
from sklearn.model_selection import cross_val_score
counter = 1
Model_Name = 'DNN_model'

with mlflow.start_run():
    
    # fit the model
    start = datetime.now()
    history_model = model.fit(train_dataset,
                            steps_per_epoch=len(train_dataset),
                            validation_data=valid_dataset,
                            validation_steps=int(0.25*len(valid_dataset)),
                            callbacks=[early_stopping, reduce_lr],
                            epochs=epoch) 
    end = datetime.now()
    print(f"The time taken to train the model is {end - start}")
    
    # Evaluate model
    model.evaluate(test_dataset)
    
    # Calculate the metrics
    model_results = eval_metrics(model, 
                                 test_features = test_feature,
                                 test_labels = test_label)
    
    mlflow.log_param("Model"           , Model_Name)
    mlflow.log_params(model_results)
    mlflow.tensorflow.autolog()
    
    signature = infer_signature(valid_feature, classifier.predict(valid_feature))
    if av_precision > 0.95 :
        mlflow.sklearn.log_model(classifier,Model_Name, signature=signature)
        print(f"f1 socre is more than 0.945 so the {Model_Name} is saved")
    else :
        print(f"Because f1 socre is not quality. The model is skip to saving phase.")
    
    print("________________________________________")

Epoch 1/25
 15011/910203 [..............................] - ETA: 1:39:55 - loss: 1.0568 - accuracy: 0.5451

KeyboardInterrupt: 